In [ ]:
from bs4 import BeautifulSoup
import gc
import json
import numpy as np
import pandas as pd
import requests
import time
from tqdm import tqdm_notebook

Task:
Webscrape Lamudi

Output:
1. Webscraper Codes
2. Lamudi Dataset
3. Lamudi Data Dictionary
4. One slider information about Lamudi

Expected Columns (but not limited to):
1. OFFER_TYPE – Buy, Rent
2. PROP_TYPE – Land, House, Condominium, Apartment, Commercial
3. TITLE
4. ADDRESS
5. PROVINCE
6. MUNICITY
7. BARANGAY
8. COORDINATES
9. PRICE
10. RATING
11. LISTING LINK

AMENITIES NOT LIMITED TO:

12. BEDROOM
13. BATHROOM
13. FLOOR AREA
14. LAND SIZE

## Webscraping via:
1. Frontend
2. Backend

Tools needed:
1. Familiarity w Python
2. BeautifulSoup Module


# Frontend
1. Familiarity with html tags
2. Web browser inspect tool

### HTML tags:
![image.png](attachment:image.png)


Southstar drug

In [ ]:
#extraction
south="https://southstardrug.com.ph/pages/store-locations" #url
south_response = requests.get(south) #get request
s_c = south_response.content #request content

In [ ]:
soup = BeautifulSoup(s_c, "html.parser") #parse the content
soup

<!DOCTYPE html>

<!-- Proudly powered by Shopify - https://shopify.com -->
<html class="no-js page store-locations multicurrency" lang="en">
<head>
<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet"/>
<!-- Added by AVADA SEO Suite -->
<meta content="E3lMXoXevvoZstlWh-92VL7HuTs_INsPQxMZPa2PQuo" name="google-site-verification"/>
<script>window.FPConfig = {};window.FPConfig.ignoreKeywords=["/cart","/account"]</script>
<!-- /Added by AVADA SEO Suite -->
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://southstardrug.com.ph/pages/store-locations" rel="canonical"/>
<link href="//cdn.shopify.com/s/files/1/0257/1884/9583/files/ssd_icon_2aa40769-e30b-476f-86ab-c1d2e8be22d4_32x32.png?v=1613560409" rel="shortcut icon" type="image/png"/>
<title>Store Locations | Southstar Drug</title>
<meta content="We have bra

In [ ]:
sections = soup.find_all("div",{"class":"content"})
sections

[<div class="content">
 <h4>NATIONAL CAPITAL REGION</h4>
 <h6>CALOOCAN CITY</h6>
 <hr class="rounded"/>SSD RSC METRO PLAZA MALL MALARIA<br/>Ph 148-01 Metro Plaza Mall, Quirino Highway, Malaria, Caloocan City<br/>Store Hours: 7:00AM - 9:00PM <br/><hr class="dashed"/>SSD R VICTORY MALL-CALOOCAN<br/>Ground Floor Victory Central Mall, Caloocan City<br/>+632-82833778<br/>Store Hours: <br/><hr class="dashed"/>SSD CALOOCAN BAGONG SILANG<br/>Phase 1 Package 1 Block 28 Lot 3638 Bagong Silang Caloocan City<br/>+632-83722847<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD CALOOCAN SANGANDAAN<br/>#1107 General San Miguel Sangandaan, Caloocan City<br/>+632-89617601<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD CALOOCAN BAGONG SILANG BARANGAY 176<br/>Ph1 Pkg3 B44 L33 Brgy 176 Zone 15, District 1 Bagong Silang, Caloocan City<br/>+63923-744-2165<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD PRIMARK DEPARO CALOOCAN<br/>G/F Lc G02, Barrio Road 171, Deparo Caloocan Ci

In [ ]:
sections[0]

<div class="content">
<h4>NATIONAL CAPITAL REGION</h4>
<h6>CALOOCAN CITY</h6>
<hr class="rounded"/>SSD RSC METRO PLAZA MALL MALARIA<br/>Ph 148-01 Metro Plaza Mall, Quirino Highway, Malaria, Caloocan City<br/>Store Hours: 7:00AM - 9:00PM <br/><hr class="dashed"/>SSD R VICTORY MALL-CALOOCAN<br/>Ground Floor Victory Central Mall, Caloocan City<br/>+632-82833778<br/>Store Hours: <br/><hr class="dashed"/>SSD CALOOCAN BAGONG SILANG<br/>Phase 1 Package 1 Block 28 Lot 3638 Bagong Silang Caloocan City<br/>+632-83722847<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD CALOOCAN SANGANDAAN<br/>#1107 General San Miguel Sangandaan, Caloocan City<br/>+632-89617601<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD CALOOCAN BAGONG SILANG BARANGAY 176<br/>Ph1 Pkg3 B44 L33 Brgy 176 Zone 15, District 1 Bagong Silang, Caloocan City<br/>+63923-744-2165<br/>Store Hours: 7:00AM-7:00PM<br/><hr class="dashed"/>SSD PRIMARK DEPARO CALOOCAN<br/>G/F Lc G02, Barrio Road 171, Deparo Caloocan City<b

In [ ]:
#cleaning process
def clean_extracted_data(sections):
    import re
    s=[]
    pattern = re.compile(r"^09\d{2}-?")
    for item in str(sections).replace("\n","").replace('<hr class="rounded"/>','<br/>').replace('<hr class="dashed"/>','<br/>').split("<br/>")[1:-1]:
        if (item == '') or ("<h6>" in item) or ("Store Hours" in item) or ("Store hours" in item) or  ("+63" in item) or (pattern.match(item)) or ("Store Location:" in item) or ("PUEBLO VERDE SUBD., BRGY. BASAK, LAPULAPU CITY, CEBU" in item):
            continue
        s.append(item)

    gc.collect()
    return s

def get_branch_address_lists(raw_list):
    stores_branch = []
    stores_address = []

    for i in range(len(raw_list)):
        if i %2 ==0:
            stores_branch.append(raw_list[i])
        else:
            stores_address.append(raw_list[i])

    stores_df = pd.DataFrame({"BRANCH": stores_branch,
                              "ADDRESS": stores_address})
    
    gc.collect()
    return stores_df

In [ ]:
ss_df = (pd.concat([get_branch_address_lists(raw_list) for raw_list in tqdm_notebook([clean_extracted_data(section) for section in sections])])
           .drop_duplicates()
           .assign(NAME = "SOUTHSTAR DRUG",
                   ADDRESS = lambda x: x["ADDRESS"].str.upper().str.replace("SSD ",""),
                   COMPLETE_ADD = lambda x: x["NAME"]+" "+x["ADDRESS"])
                   [["NAME","ADDRESS","COMPLETE_ADD"]])

display(ss_df.info(), ss_df.head())

C:\Users\trdfrago\AppData\Local\Temp/ipykernel_18320/2022015389.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  ss_df = (pd.concat([get_branch_address_lists(raw_list) for raw_list in tqdm_notebook([clean_extracted_data(section) for section in sections])])


  0%|          | 0/16 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 554 entries, 0 to 7
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   NAME          554 non-null    object
 1   ADDRESS       554 non-null    object
 2   COMPLETE_ADD  554 non-null    object
dtypes: object(3)
memory usage: 17.3+ KB


None

,NAME,ADDRESS,COMPLETE_ADD
0,SOUTHSTAR DRUG,"PH 148-01 METRO PLAZA MALL, QUIRINO HIGHWAY, M...","SOUTHSTAR DRUG PH 148-01 METRO PLAZA MALL, QUI..."
1,SOUTHSTAR DRUG,"GROUND FLOOR VICTORY CENTRAL MALL, CALOOCAN CITY",SOUTHSTAR DRUG GROUND FLOOR VICTORY CENTRAL MA...
2,SOUTHSTAR DRUG,PHASE 1 PACKAGE 1 BLOCK 28 LOT 3638 BAGONG SIL...,SOUTHSTAR DRUG PHASE 1 PACKAGE 1 BLOCK 28 LOT ...
3,SOUTHSTAR DRUG,"#1107 GENERAL SAN MIGUEL SANGANDAAN, CALOOCAN ...",SOUTHSTAR DRUG #1107 GENERAL SAN MIGUEL SANGAN...
4,SOUTHSTAR DRUG,"PH1 PKG3 B44 L33 BRGY 176 ZONE 15, DISTRICT 1 ...",SOUTHSTAR DRUG PH1 PKG3 B44 L33 BRGY 176 ZONE ...


Puregold

In [ ]:
puregold_link = "http://puregoldbeta.webtogo.com.ph/store_locator.do?regionkey=Bicol"

puregold_response = requests.get(puregold_link)
content = puregold_response.content
soup = BeautifulSoup(content, "html.parser")
search_results = soup.find_all("td",{"class":"storeContent"})

search_results

[<td class="storeContent">
 <strong>PPCI-EMBARCADERO</strong>
 <p>
 	Brgy. Victory Village, Legazpi, Albay</p>
 <p>
 	 </p>
 <p>
 	Business Hours:</p>
 <p>
 	9:00am-9:00pm</p>
 <span class="telno"><p>
 <span style="font-family: 'Wingdings 2'; font-size: 11px; line-height: 12px; ">'</span> 820 1452</p>
 </span>
 </td>,
 <td class="storeContent">
 <strong>PPCI-NAGA</strong>
 <p>
 	GF ALDP Mall Brgy. Triangulo, Naga City</p>
 <p>
 	 </p>
 <p>
 	Business Hours: 9:00am - 9:00pm</p>
 <span class="telno"><p>
 <span style="font-family: 'Wingdings 2'; font-size: 11px; line-height: 12px; ">'</span> 250-81-90</p>
 </span>
 </td>,
 <td class="storeContent">
 <strong>PPCI-CENTRO NAGA</strong>
 <p>
 	LBDRC Bldg. General Luna St.,Naga City</p>
 <p>
 	 </p>
 <p>
 	Business Hours : 8:00am - 9:00pm</p>
 <span class="telno"><p>
 <span font-size:="" lang="EN-US" mso-ansi-language:="" mso-bidi-font-family:="" mso-bidi-language:="" mso-fareast-font-family:="" mso-fareast-language:="" mso-fareast-theme-font:

In [ ]:
def clean_entry(item,tag):
    text_content = ""
    try:
        text_content = item.find(tag).text.strip().upper().split('\r')[0]
    except:
        pass
    return text_content

In [ ]:
Regions = ["Bicol", "NCR+Metro%20Manila",
           "Ilocos+Region", "CAR+Cordillera%20Administrative%20Region",
           "Central+Luzon", "CALABARZON","Cagayan+Valley%20Region",
           "MIMAROPA", "Western+Visayas%20Region", "Southern+Mindanao%20Region",
           "Northern+Mindanao%20Region", "Caraga+Region"]

puregold_links = [f"http://puregoldbeta.webtogo.com.ph/store_locator.do?regionkey={region}" for region in Regions]

puregold_branch_list = []
puregold_address_list = []
for puregold_link in tqdm_notebook(puregold_links):
    puregold_response = requests.get(puregold_link)
    content = puregold_response.content
    soup = BeautifulSoup(content, "html.parser")
    search_results = soup.find_all("td",{"class":"storeContent"})

    branch_list = list(filter(None,[clean_entry(item, "strong") for item in search_results]))
    address_list = list(filter(None,[clean_entry(item, "p") for item in search_results]))

    puregold_branch_list.append(branch_list)
    puregold_address_list.append(address_list)

puregold_branches=sum(puregold_branch_list, [])
puregold_addresses=sum(puregold_address_list, [])

rename_dict = {"PPCI":"PUREGOLD",
               "PG":"PUREGOLD"}

puregold_df = (pd.DataFrame({"BRANCH": puregold_branches,
                            "ADDRESS": puregold_addresses})
                 .assign(BRANCH = lambda x: x["BRANCH"].replace(rename_dict, regex=True),
                         COMPLETE_ADD = lambda x: x["BRANCH"]+", "+x["ADDRESS"]))

display(puregold_df.info(), puregold_df.head())

C:\Users\trdfrago\AppData\Local\Temp/ipykernel_20136/814906961.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for puregold_link in tqdm_notebook(puregold_links):


  0%|          | 0/12 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BRANCH        183 non-null    object
 1   ADDRESS       183 non-null    object
 2   COMPLETE_ADD  183 non-null    object
dtypes: object(3)
memory usage: 4.4+ KB


None

,BRANCH,ADDRESS,COMPLETE_ADD
0,PUREGOLD-EMBARCADERO,"BRGY. VICTORY VILLAGE, LEGAZPI, ALBAY","PUREGOLD-EMBARCADERO, BRGY. VICTORY VILLAGE, L..."
1,PUREGOLD-NAGA,"GF ALDP MALL BRGY. TRIANGULO, NAGA CITY","PUREGOLD-NAGA, GF ALDP MALL BRGY. TRIANGULO, N..."
2,PUREGOLD-CENTRO NAGA,"LBDRC BLDG. GENERAL LUNA ST.,NAGA CITY","PUREGOLD-CENTRO NAGA, LBDRC BLDG. GENERAL LUNA..."
3,PUREGOLD - LEGAZPI ALBAY,"BRGY. OF VICTORY VILLAGE, CITY OF LEGAZPI, ALBAY","PUREGOLD - LEGAZPI ALBAY, BRGY. OF VICTORY VIL..."
4,PUREGOLD - NAGA CAMARINES SUR,"BRGY. TRIANGULO, NAGA CITY","PUREGOLD - NAGA CAMARINES SUR, BRGY. TRIANGULO..."


# Backend
1. Network tab in Web browser inspect tool

Rose pharmacy

In [ ]:
rose="https://www.rosepharmacy.com/ph1/wp-content/plugins/superstorefinder-wp/ssf-wp-xml.php?wpml_lang=&t=1683506140134"
rose_response = requests.get(rose)
r_c = rose_response.content
rsoup = BeautifulSoup(r_c, "html.parser") 

In [ ]:
rose_branches_dict = [{"BRANCH":item.find("location").text,
                       "ADDRESS":item.find("address").text.replace("&#44;",","),
                       "LATITUDE":float(item.find("latitude").text),
                       "LONGITUDE":float(item.find("longitude").text)} for item in rsoup.find_all("item")]

In [ ]:
rose_df = (pd.DataFrame.from_dict(rose_branches_dict)
                       .drop_duplicates()
                       .assign(NAME = "ROSE PHARMACY",
                               ADDRESS = lambda x: x["ADDRESS"].str.upper(),
                               COMPLETE_ADD = lambda x: x["NAME"]+", "+x["ADDRESS"])
                       [["NAME","ADDRESS","LATITUDE","LONGITUDE","COMPLETE_ADD"]])

display(rose_df)

,NAME,ADDRESS,LATITUDE,LONGITUDE,COMPLETE_ADD
0,ROSE PHARMACY,"REAL ST. BRGY NALIBUNAN ABUYOG, LEYTE ABUYOG,...",10.750271,125.008428,"ROSE PHARMACY, REAL ST. BRGY NALIBUNAN ABUYOG,..."
1,ROSE PHARMACY,"HI-WAY ARGAO CEBU ARGAO, CEBU 6021",9.932581,123.550408,"ROSE PHARMACY, HI-WAY ARGAO CEBU ARGAO, CEBU..."
2,ROSE PHARMACY,"PRINCE WAREHOUSE ARGAO, DR. T.S KINTANAR ST., ...",9.882991,123.600381,"ROSE PHARMACY, PRINCE WAREHOUSE ARGAO, DR. T.S..."
3,ROSE PHARMACY,"POBLACION ASTURIAS, CEBU ASTURIAS, CEBU 6042",10.567295,123.721036,"ROSE PHARMACY, POBLACION ASTURIAS, CEBU ASTUR..."
4,ROSE PHARMACY,"BRGY. SAN JOSE , AURORA , ZAMBOANGA DEL SUR A...",7.960197,123.580406,"ROSE PHARMACY, BRGY. SAN JOSE , AURORA , ZAMBO..."
...,...,...,...,...,...
321,ROSE PHARMACY,"COL. MARIANO GARCES ST. POBLACION, UBAY BOHOL ...",10.057204,124.472952,"ROSE PHARMACY, COL. MARIANO GARCES ST. POBLACI..."
322,ROSE PHARMACY,"G. LAVINIA AVENUE, VALENCIA CITY, BUKIDNON VA...",7.904879,125.090623,"ROSE PHARMACY, G. LAVINIA AVENUE, VALENCIA CIT..."
323,ROSE PHARMACY,"MAGSAYSAY COR. MABINI STS., VALENCIA, BUKIDNON...",7.904424,125.090316,"ROSE PHARMACY, MAGSAYSAY COR. MABINI STS., VAL..."
324,ROSE PHARMACY,"MAGSAYSAY ST., POBLACION, VALENCIA, BUKIDNON ...",7.903779,125.090751,"ROSE PHARMACY, MAGSAYSAY ST., POBLACION, VALEN..."


Watsons

In [ ]:
watsons_links = [f"https://api.watsons.com.ph/api/v2/wtcph/stores/watStores?currentPage={page}&pageSize=20&isCceOrCc=false&fields=FULL&lang=en&curr=PHP" for page in range(53)]

watsons_branch_list = []
for watsons_link in tqdm_notebook(watsons_links):
    content_html = requests.get(watsons_link)
    content = content_html.content
    branch_list = json.loads(content)["stores"]

    watsons_list = [{"NAME": "Watsons",
                     "ADDRESS": branch["address"]["formattedAddress"],
                     "LAT": branch["geoPoint"]["latitude"],
                     "LONG": branch["geoPoint"]["longitude"]} for branch in branch_list]
    
    watsons_branch_list.append(watsons_list)

watsons_branches=sum(watsons_branch_list, [])

watsons_df = pd.DataFrame.from_dict(watsons_branches).drop_duplicates()
watsons_df

Waltermart

In [ ]:
url = "https://api.freshop.com/1/stores?app_key=walter_mart&has_address=true&limit=-1&token=a7b3d5953b2101db0f18aa7f860885ef"
content_html = requests.get(url)
branch_list = content_html.json()["items"]
branch_list

In [ ]:
waltermart_list = [{"NAME": "Waltermart",
                    "ADDRESS": branch["address_0"]+", "+branch["address_1"]+", "+branch["address_2"],
                    "LAT": branch["latitude"],
                    "LONG": branch["longitude"]} for branch in branch_list if ("address_1" in branch) and ("address_2" in branch)]

waltermart_df = pd.DataFrame.from_dict(waltermart_list).drop_duplicates()

additional_list = [{"NAME": "Waltermart",
                    "ADDRESS": branch["name"]+", "+branch["address_0"],
                    "LAT": branch["latitude"],
                    "LONG": branch["longitude"]} for branch in branch_list if ("address_2" not in branch)]

additional_df = pd.DataFrame.from_dict(additional_list).drop_duplicates()

all_waltermart = pd.concat([waltermart_df, additional_df]).drop_duplicates()
all_waltermart


Cebuana Lhuillier

In [ ]:
def get_provinces():
    url = 'https://www.cebuanalhuillier.com/wp-admin/admin-ajax.php'
    server = requests.post(url, data={"action":"getProvinces"})
    output = server.text
    soup = BeautifulSoup(output, "html.parser") 

    province_codes = [item.find("a")["data-provincecode"] for item in soup.find_all("li")]

    gc.collect()
    return province_codes

In [ ]:
def get_branches_by_city(TownCityCode, ProvinceCode, TownCityName):
    url = 'https://www.cebuanalhuillier.com/wp-admin/admin-ajax.php'
    server = requests.post(url, data={"action":"getBranchesByCity",
                                    "TownCityCode": TownCityCode, 
                                    "ProvinceCode": ProvinceCode,
                                    "TownCityName": TownCityName})
    output = server.text
    soup = BeautifulSoup(output, "html.parser")

    branch_name_list = [item.text for item in soup.find_all("div", {"class":"branchname color-blue"})]
    branch_address_list = [item.text for item in soup.find_all("div", {"class":"branchaddress"})]

    branch_dict_list = [{"BRANCH": branch,"ADDRESS": address} for branch, address in zip(branch_name_list, branch_address_list)]
    
    gc.collect()
    return branch_dict_list

In [ ]:
def get_city_by_province(ProvinceCode):
    url = 'https://www.cebuanalhuillier.com/wp-admin/admin-ajax.php'
    server = requests.post(url, data={"action":"getCitiesByProvince", "ProvinceCode": ProvinceCode})
    output = server.text
    soup = BeautifulSoup(output, "html.parser") 

    TownCityName_list = [item.text for item in soup.find_all("option")]
    TownCityName = "".join(TownCityName_list)

    TownCityCode_list = [item["value"] for item in soup.find_all("option")]
    
    province_branches = sum([get_branches_by_city(TownCityCode, ProvinceCode, TownCityName) for TownCityCode in tqdm_notebook(TownCityCode_list)],[])

    gc.collect()
    return province_branches

In [ ]:
province_codes = get_provinces()

In [ ]:
all_branches_list = []
for ProvinceCode in tqdm_notebook(province_codes):
    print(ProvinceCode)
    all_branches_list.append(get_city_by_province(ProvinceCode))
main_branches = sum(all_branches_list,[])

C:\Users\trdfrago\AppData\Local\Temp/ipykernel_20136/3509082259.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ProvinceCode in tqdm_notebook(province_codes):


  0%|          | 0/80 [00:00<?, ?it/s]

140100000


C:\Users\trdfrago\AppData\Local\Temp/ipykernel_20136/1291162279.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  province_branches = sum([get_branches_by_city(TownCityCode, ProvinceCode, TownCityName) for TownCityCode in tqdm_notebook(TownCityCode_list)],[])


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

25700000


  0%|          | 0/6 [00:00<?, ?it/s]

45800000


  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
cb_df = (pd.DataFrame.from_dict(main_branches)
                     .drop_duplicates()
                     .assign(NAME = "CEBUHANA LHUILLIER",
                             BRANCH = lambda x: x["BRANCH"].str.replace("CLH","Cebuana Lhuillier").str.upper(),
                             ADDRESS = lambda x: x["ADDRESS"].str.upper(),
                             COMPLETE_ADD = lambda x: x["BRANCH"]+", "+x["ADDRESS"])
                     [["NAME","BRANCH","ADDRESS","COMPLETE_ADD"]])

display(cb_df.info(), cb_df.head())